### Deliverable 2a: Products Dimension table

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import re
from collections import defaultdict

In [3]:
# Load data
products = pd.read_csv("Products1.txt", delimiter="|")
product_class = pd.read_csv("product_class.csv")

In [4]:
# Create mapping dictionaries
subcategory_map = dict(
    zip(product_class['product_subcategory'], product_class['product_class_id']))

len(subcategory_map)

111

In [5]:
subcategory_list = list(subcategory_map.keys())
subcategory_list[:20]

['Nuts',
 'Shellfish',
 'Canned Fruit',
 'Spices',
 'Pasta',
 'Yogurt',
 'Coffee',
 'Deli Meats',
 'Ice Cream',
 'TV Dinner',
 'Cheese',
 'Chips',
 'Fresh Vegetables',
 'Sour Cream',
 'Cottage Cheese',
 'Deli Salads',
 'Dried Meat',
 'Paper Wipes',
 'Soda',
 'Deodorizers']

In [6]:
len(subcategory_list)

111

##### Cleaning ItemType from products

In [7]:
products['clean_itemType'] = products['itemType'].copy()


# ItemType in subcategory
valid_mask = products['clean_itemType'].isin(subcategory_map.keys())
print(valid_mask.sum())

1186


##### Fuzzy matching for similar but not identical types

In [8]:
def find_best_match(item_type):
    if pd.isna(item_type):
        return None
    
    if item_type in subcategory_map:
        return item_type
    

    # common subsitutions
    common_subs = {
        'snacks': "Chips",
        "bread": "Sliced Bread",
        "drink": "Beverages",
        "frozen": "Frozen Foods",
        "meats": "Deli Meats"
    }

    for old, new in common_subs.items():
        if old is str(item_type).lower():
            return new
        

    # Fuzzing matching
    match, score = process.extractOne(
        str(item_type), subcategory_list, scorer=fuzz.token_set_ratio)

    return match if score > 65 else None

In [9]:
needed_mapping = ~valid_mask & products['clean_itemType'].notna()
print(needed_mapping.sum())

595


##### Matching only to non-valid types

In [10]:
products.loc[needed_mapping, 'clean_itemType'] = (
    products.loc[needed_mapping, 'clean_itemType'].apply(find_best_match).values
)

In [11]:
products.sample(5)

,Manufacturer,Product Name,Size,itemType,SKU,BasePrice,clean_itemType
989,Hilltop,Hilltop Mint Mouthwash,16.2,Mouthwash,43075001,$3.62,Mouthwash
1955,Beech-Nut,Stage 2 Yellow Cling Peaches,4 oz,Baby Food,44041001,$0.79,Baby Food
997,High Top,High Top Cauliflower,17.7,Fresh Vegetables,43083001,$2.23,Fresh Vegetables
960,Hostess,Cake Ding Dongs,15.3 oz,NaN,43044001,$3.89,NaN
1207,Gerber,1st Foods Prunes,5 oz,Baby Food,43293001,$1.09,Baby Food


##### Product name keyword analysis for nulls

In [12]:
keyword_map = defaultdict(list)

for subcat in subcategory_list:
    keywords = re.sub(r'([A-Z])', r' \1', subcat).split()
    keywords.extend(subcat.lower().split('_'))
    keywords.extend(subcat.lower().split())

    for kw in set(keywords):
        if len(kw) > 3:  # ignores short keywords
            keyword_map[kw].append(subcat)


keyword_map

defaultdict(list,
            {'Nuts': ['Nuts'],
             'nuts': ['Nuts'],
             'shellfish': ['Shellfish'],
             'Shellfish': ['Shellfish'],
             'canned fruit': ['Canned Fruit'],
             'canned': ['Canned Fruit', 'Canned Vegetables'],
             'Fruit': ['Canned Fruit', 'Fresh Fruit', 'Dried Fruit'],
             'fruit': ['Canned Fruit', 'Fresh Fruit', 'Dried Fruit'],
             'Canned': ['Canned Fruit', 'Canned Vegetables'],
             'spices': ['Spices'],
             'Spices': ['Spices'],
             'Pasta': ['Pasta'],
             'pasta': ['Pasta'],
             'yogurt': ['Yogurt'],
             'Yogurt': ['Yogurt'],
             'Coffee': ['Coffee'],
             'coffee': ['Coffee'],
             'Deli': ['Deli Meats', 'Deli Salads'],
             'deli': ['Deli Meats', 'Deli Salads'],
             'Meats': ['Deli Meats'],
             'meats': ['Deli Meats'],
             'deli meats': ['Deli Meats'],
             'Cream': ['Ice 

In [13]:
def keyword_analyzer(product_name):
    if pd.isna(product_name):
        return None
    
    product_name = str(product_name).lower()
    matches = []

    # Brand mappings
    brand_keywords = {
        'frito': 'Chips',
        'lays': 'Chips',
        'dannon': 'Yogurt',
        'yoplait': 'Yogurt',
        'kraft': 'Cheese',
        'tropicana': 'Juice'
    }

    for brand, subcat in brand_keywords.items():
        if brand in product_name:
            return subcat
        
    
    # product keyword matching
    for kw in keyword_map:
        if kw in product_name:
            matches.extend(keyword_map[kw])
    
    return max(set(matches), key=matches.count) if matches else None

##### Applying keyword analysis to remaining nulls

In [14]:
still_null = products['clean_itemType'].isna()
print(still_null.sum())

399


In [15]:
products.loc[still_null, 'clean_itemType'] = (products.loc[still_null, 'Product Name'].apply(keyword_analyzer).values)

In [16]:
products.sample(5)

,Manufacturer,Product Name,Size,itemType,SKU,BasePrice,clean_itemType
1642,Cutting Edge,Cutting Edge Pimento Loaf,9.5,Bologna,43728001,$1.82,Bologna
1823,Booker,Booker String Cheese,18.6,Cheese,43909001,$1.66,Cheese
705,Luvs,With Ultra Leakguards Size 4,88 ct,Diapers,42788001,$18.99,Diapers
1846,Bird Call,Bird Call Childrens Aspirin,17.8,Aspirin,43932001,$2.87,Aspirin
629,Nabisco,Wheat Thins Hint Of Salt,9.1 oz,NaN,42710001,$2.39,None


In [17]:
products.isna().sum()

Manufacturer        0
Product Name        0
Size                0
itemType          294
SKU                 0
BasePrice           0
clean_itemType    144
dtype: int64

In [18]:
if products['clean_itemType'].isna().any():
    most_common = products['clean_itemType'].mode()[0]
    products['clean_itemType'].fillna(most_common, inplace=True)
    print(
        f"Assigned {products['clean_itemType'].isna().sum()} remaining products to '{most_common}'")


Assigned 0 remaining products to 'Baby Food'


C:\Users\pavan\AppData\Local\Temp\ipykernel_16872\667822496.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  products['clean_itemType'].fillna(most_common, inplace=True)


##### Creating Product Dimension table
+ Merging Products with product class using **clean_itemType** and **product_subcategory** 

In [19]:
product_dim = products.merge(product_class, left_on='clean_itemType', right_on="product_subcategory", how="left")
product_dim.sample(5)

,Manufacturer,Product Name,Size,itemType,SKU,BasePrice,clean_itemType,product_class_id,product_subcategory,product_category,product_department,product_family
638,Pampers,Swaddlers Size 1,35 ct,Diapers,42618001,$10.49,Diapers,213,Diapers,Paper Products,Household,Non-Consumable
648,Oscar Mayer,Hot Dogs Turkey Smoked,16 oz,NaN,42628001,$2.49,Hot Dogs,81,Hot Dogs,Meat,Deli,Food
1451,Gerber,Stage 2 Pears,4 oz,Baby Food,43344001,$0.69,Baby Food,209,Baby Food,Baby Food,Baby Food,Food
1183,High Top,High Top Shitake Mushrooms,11.3,Fresh Vegetables,43112001,$0.69,Fresh Vegetables,61,Fresh Vegetables,Vegetables,Produce,Food
1403,Gerber,Cereal Multigrain,8 oz,Baby Food,43296001,$2.49,Baby Food,209,Baby Food,Baby Food,Baby Food,Food


In [20]:
product_dim.shape

(2344, 12)

In [21]:
products['clean_itemType'].value_counts()

clean_itemType
Baby Food           313
Sauces              166
Coffee              134
Fresh Vegetables    120
Cereal               93
                   ... 
Shower Soap           1
Sponges               1
Crackers              1
Bananas               1
Sardines              1
Name: count, Length: 66, dtype: int64

##### Handling merge nulls

In [22]:
product_dim['product_class_id'] = product_dim['product_class_id'].fillna(-1)
product_dim.sample(5)

,Manufacturer,Product Name,Size,itemType,SKU,BasePrice,clean_itemType,product_class_id,product_subcategory,product_category,product_department,product_family
140,Tell Tale,Tell Tale Cauliflower,13.6,Fresh Vegetables,42192001,$1.87,Fresh Vegetables,13,Fresh Vegetables,Vegetables,Produce,Food
387,Red Spade,Red Spade Roasted Chicken,20.7,Fresh Chicken,42378001,$2.76,Fresh Chicken,77,Fresh Chicken,Meat,Deli,Food
2043,Bushs,Brown Sugar Hickory Baked Beans,28 oz,Canned Goods,43860001,$1.79,Canned Fruit,3,Canned Fruit,Fruit,Canned Products,Food
1139,High Top,High Top Cauliflower,17.7,Fresh Vegetables,43083001,$2.23,Fresh Vegetables,13,Fresh Vegetables,Vegetables,Produce,Food
67,Tri-State,Tri-State Party Nuts,12.6,Nuts,42132001,$2.63,Nuts,1,Nuts,Specialty,Produce,Food


In [23]:
for col in ['product_category', 'product_department', 'product_family']:
    product_dim[col] = product_dim[col].fillna("Uncategorized")

In [24]:
product_dim.reset_index(drop=True, inplace=True)

In [25]:
counter = 1
def generate_unique_key(sku):
    """Generate a unique key for the given SKU."""
    global counter
    key = counter
    counter += 1
    return key

In [26]:
product_keys = product_dim['SKU'].apply(generate_unique_key).values



new_products = pd.DataFrame({
    'ProductKey': product_keys,
    'SKU': product_dim['SKU'],
    'ProductName': product_dim['Product Name'],
    'ProductClass ID': product_dim['product_class_id'],
    'Subcategory': product_dim['clean_itemType'],
    'Category': product_dim['product_category'],
    'Department': product_dim['product_department'],
    'ProductFamily': product_dim['product_family'],
    'Size': product_dim['Size'],
    '#PerCase': np.full(len(product_dim), 12),  # Fixed length array
    'BrandName': product_dim['Manufacturer'],
    'Manufacturer': product_dim['Manufacturer'],
    'Supplier': np.where(
        product_dim['Product Name'].str.contains(
            'Milk', case=False, na=False),
        'Rowan Dairy',
        'Rowan Warehouse'
    )
})

new_products.head(10)

,ProductKey,SKU,ProductName,ProductClass ID,Subcategory,Category,Department,ProductFamily,Size,#PerCase,BrandName,Manufacturer,Supplier
0,1,42081001,Jambalaya Rice Mix,57,Rice,Starchy Foods,Starchy Foods,Food,12 oz,12,Zatarains,Zatarains,Rowan Warehouse
1,2,42082001,Jambalaya Rice Mix,57,Rice,Starchy Foods,Starchy Foods,Food,8 oz,12,Zatarains,Zatarains,Rowan Warehouse
2,3,42083001,Guacamole Regular,209,Baby Food,Baby Food,Baby Food,Food,8 oz,12,Yucatan,Yucatan,Rowan Warehouse
3,4,42084001,Coffee Original Blend,7,Coffee,Dry Goods,Baking Goods,Drink,12 oz,12,Yuban,Yuban,Rowan Warehouse
4,5,42084001,Coffee Original Blend,90,Coffee,Hot Beverages,Beverages,Drink,12 oz,12,Yuban,Yuban,Rowan Warehouse
5,6,42085001,GoGurt Variety Pack,6,Yogurt,Dairy,Dairy,Food,8 ct,12,Yoplait,Yoplait,Rowan Warehouse
6,7,42086001,Italian Dressing,16,Deli Salads,Side Dishes,Deli,Food,16 oz,12,Wishbone,Wishbone,Rowan Warehouse
7,8,42087001,Cheeseburger Heat & Serve Sliders,15,Cottage Cheese,Dairy,Dairy,Food,29.28 oz,12,White Castle,White Castle,Rowan Warehouse
8,9,42088001,Choice Cuts Poultry,214,Pet Food,Pet Supplies,Household,Non-Consumable,36 oz,12,Whiskas,Whiskas,Rowan Warehouse
9,10,42089001,Farmers Pick Concord Grape,209,Baby Food,Baby Food,Baby Food,Food,46 oz,12,Welchs,Welchs,Rowan Warehouse


In [27]:
new_products.isna().sum()

ProductKey         0
SKU                0
ProductName        0
ProductClass ID    0
Subcategory        0
Category           0
Department         0
ProductFamily      0
Size               0
#PerCase           0
BrandName          0
Manufacturer       0
Supplier           0
dtype: int64

In [28]:
new_products.shape

(2344, 13)

In [29]:
import sqlite3 as lite

In [30]:
conn = lite.connect("store1.db")
curr = conn.cursor()

In [31]:
curr.execute("DROP TABLE IF EXISTS ProductDimension")

curr.execute("""
    CREATE TABLE ProductDimension (
    ProductKey INTEGER PRIMARY KEY AUTOINCREMENT,
    SKU VARCHAR(50) NOT NULL,
    ProductName VARCHAR(255) NOT NULL,
    ProductClassID INT NOT NULL,
    Subcategory VARCHAR(100) NOT NULL,
    Category VARCHAR(100) NOT NULL,
    Department VARCHAR(100) NOT NULL,
    ProductFamily VARCHAR(100) NOT NULL,
    Size VARCHAR(50),
    PerCase INT NOT NULL,
    BrandName VARCHAR(100),
    Manufacturer VARCHAR(100),
    Supplier VARCHAR(100) NOT NULL
    );

""")

In [32]:
new_products.to_sql("ProductDimension", conn, if_exists="replace", index=False)

2344

In [33]:
curr.execute("Select * from ProductDimension limit 10")

for row in curr.fetchall():
    print(row)

(1, 42081001, 'Jambalaya Rice Mix', 57, 'Rice', 'Starchy Foods', 'Starchy Foods', 'Food', '12 oz', 12, 'Zatarains', 'Zatarains', 'Rowan Warehouse')
(2, 42082001, 'Jambalaya Rice Mix', 57, 'Rice', 'Starchy Foods', 'Starchy Foods', 'Food', '8 oz', 12, 'Zatarains', 'Zatarains', 'Rowan Warehouse')
(3, 42083001, 'Guacamole Regular', 209, 'Baby Food', 'Baby Food', 'Baby Food', 'Food', '8 oz', 12, 'Yucatan', 'Yucatan', 'Rowan Warehouse')
(4, 42084001, 'Coffee Original Blend', 7, 'Coffee', 'Dry Goods', 'Baking Goods', 'Drink', '12 oz', 12, 'Yuban', 'Yuban', 'Rowan Warehouse')
(5, 42084001, 'Coffee Original Blend', 90, 'Coffee', 'Hot Beverages', 'Beverages', 'Drink', '12 oz', 12, 'Yuban', 'Yuban', 'Rowan Warehouse')
(6, 42085001, 'GoGurt Variety Pack', 6, 'Yogurt', 'Dairy', 'Dairy', 'Food', '8 ct', 12, 'Yoplait', 'Yoplait', 'Rowan Warehouse')
(7, 42086001, 'Italian Dressing', 16, 'Deli Salads', 'Side Dishes', 'Deli', 'Food', '16 oz', 12, 'Wishbone', 'Wishbone', 'Rowan Warehouse')
(8, 42087001, 

In [34]:
new_products.to_csv("ProductDimension-final.csv", index=False)

##### Deliverable 2b Implementation: Metadata Tracking and Date Standardization

##### Metadata Tracking System

In [35]:
source_definition = pd.DataFrame({
    'source_id': [1, 2, 3, 4, 5],
    'source_description': [
        'Original product table',
        'Manually mapped by Pavan Kumar Mistry',
        'String match (e.g., Product Name contains "Frito Lay" → "Chips")',
        'Fuzzy matched from itemType',
        'Keyword analysis from Product Name'
    ],
    'responsible_party': [
        'ETL System',
        'Pavan Kumar Mistry',
        'Automated Matcher',
        'Fuzzy Matching Algorithm',
        'Keyword Analyzer'
    ]

})

In [36]:
valid_mask = product_dim['clean_itemType'].isin(subcategory_map.keys())
needed_mapping = ~valid_mask & products['clean_itemType'].notna()
still_null = product_dim['clean_itemType'].isna()

In [37]:
all_in_map = product_dim['clean_itemType'].apply(
    lambda x: x in subcategory_map.keys()).all()

all_in_map

np.True_

In [38]:
if all_in_map:
    # tracking based on whether itemType was modified
    product_sources = pd.DataFrame({
        'SKU': product_dim['SKU'],
        'source_id': np.where(
            product_dim['itemType'] == product_dim['clean_itemType'],
            1,  # original matched exactly
            np.where(
                product_dim['itemType'].isna(),
                4,  # came from keyword analysis
                3   # was modified through fuzzy/string matching
            )
        ),
        'mapping_date': pd.Timestamp.now().strftime('%Y%m%d')
    })
else:
    # Original logic if not all values are in the map
    valid_mask = product_dim['clean_itemType'].isin(subcategory_map.keys())
    needed_mapping = ~valid_mask & product_dim['clean_itemType'].notna()
    still_null = product_dim['clean_itemType'].isna()

    product_sources = pd.DataFrame({
        'SKU': product_dim['SKU'],
        'source_id': np.where(
            valid_mask, 1,
            np.where(
                needed_mapping, 3,  # modified though fuzzy/string matching
                np.where(
                    still_null, 4,  # using keyword analysis
                    2  # Manually
                )
            )
        ),
        'mapping_date': pd.Timestamp.now().strftime('%Y%m%d')
    })

In [39]:
# Test
assert len(product_sources) == len(product_dim), "Lengths still don't match!"

In [40]:
# Create tables
curr.execute("DROP TABLE IF EXISTS SourceDefinitions")
curr.execute("""
    CREATE TABLE SourceDefinitions (
        source_id INT PRIMARY KEY,
        source_description TEXT NOT NULL,
        responsible_party TEXT NOT NULL
    );
""")

In [41]:
source_definition.to_sql("SourceDefinitions", conn, if_exists="replace", index=False)

5

In [42]:
res = curr.execute("select * from SourceDefinitions")
rows = curr.fetchall()
for row in rows:
    print(row)

(1, 'Original product table', 'ETL System')
(2, 'Manually mapped by Pavan Kumar Mistry', 'Pavan Kumar Mistry')
(3, 'String match (e.g., Product Name contains "Frito Lay" → "Chips")', 'Automated Matcher')
(4, 'Fuzzy matched from itemType', 'Fuzzy Matching Algorithm')
(5, 'Keyword analysis from Product Name', 'Keyword Analyzer')


In [43]:
curr.execute("DROP TABLE IF EXISTS ProductMetaData")
curr.execute("""
    CREATE TABLE ProductMetaData (
        SKU VARCHAR(50) NOT NULL,
        source_id INT NOT NULL,
        mapping_date CHAR(8) NOT NULL,
        PRIMARY KEY (SKU, source_id),
        FOREIGN KEY (source_id) REFERENCES SourceDefinitions(source_id),
        FOREIGN KEY (SKU) REFERENCES ProductDimension(SKU)
    );
""")

In [44]:
product_sources.to_sql("ProductMetaData", conn, if_exists='replace', index=False)

2344

In [45]:
product_sources.to_csv("ProductMetaData-final.csv", index=False)

In [46]:
res = curr.execute("select * from ProductMetaData")

product_meta_data = pd.DataFrame(res, columns=["SKU", "source_id", "mapping_date"])
product_meta_data.sample(10)

,SKU,source_id,mapping_date
2172,43989001,1,20250328
681,42655001,4,20250328
379,42370001,1,20250328
83,42142001,1,20250328
819,42775001,3,20250328
1581,43474001,3,20250328
978,42928001,3,20250328
697,42667001,3,20250328
438,42428001,3,20250328
1334,43231001,1,20250328


In [47]:
product_sources['source_id'].value_counts()

source_id
1    1321
3     729
4     294
Name: count, dtype: int64

In [48]:
curr.close()
conn.close()